<a href="https://colab.research.google.com/github/ducvu/ml_projects/blob/master/newbornweight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

filename = '/content/data_all_update.xlsx'
df = pd.read_excel(filename)

In [157]:
print('-------------------------------------')
print('Columns : ',df.columns)

print('-------------------------------------')
print("NB null values = ",df.isnull().sum().max())

print('-------------------------------------')
print('Head : ', df.head(5))

print('-------------------------------------')
print( df.info())

print('-------------------------------------')
print(df.describe())
print('-------------------------------------')

-------------------------------------
Columns :  Index(['năm sinh', 'Cân nặng', 'Chiều cao', 'BMI', 'Tuần thai', 'BC', 'CHBC',
       'Hct', 'Hb', 'MCV', 'MCH', 'MCHC', 'Mao mạch', 'BPD', 'AC', 'FL',
       'Sexe', 'Weight'],
      dtype='object')
-------------------------------------
NB null values =  351
-------------------------------------
Head :     năm sinh  Cân nặng  Chiều cao    BMI  ...     AC    FL    Sexe  Weight
0      1982      63.0       1.50  28.00  ...  350.0  74.0    Male    3600
1      1999      46.0       1.59  28.90  ...    NaN   NaN  Female     350
2      2001      45.0       1.53  19.00  ...    NaN   NaN    Male     800
3      1998      47.0       1.45  22.35  ...    NaN   NaN    Male     900
4      1992      45.0       1.45  21.40  ...  241.0  49.0  Female    1200

[5 rows x 18 columns]
-------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dt

In [152]:
df["BC"].replace({"8..12": "8.12"}, inplace=True)
df["Hb"].replace({".36.7": "36.7"}, inplace=True)
df["Hb"].replace({".36.5": "36.5"}, inplace=True)
df["MCHC"].replace({".35.9": "35.9"}, inplace=True)

In [153]:
# select all numerical cols and fill missing values by mean of column
#df["BC"] = df["BC"].astype('float64')
#df["Hb"] = df["Hb"].astype('float64')
#df["MCHC"] = df["MCHC"].astype('float64')
df_num = df.select_dtypes(include=['float64','int64']).copy()
df_num.head()
df_num = df_num.fillna(df_num.mean())
print(df_num.head())

df_num['Sexe'] = df['Sexe']

# select all categorial cols and fill missing values by "MISSING" 
df_obj = df.select_dtypes(include=['object']).copy()
print(df_obj.head())


   năm sinh  Cân nặng  Chiều cao  ...          AC          FL  Weight
0      1982      63.0       1.50  ...  350.000000   74.000000    3600
1      1999      46.0       1.59  ...  317.080258  148.144044     350
2      2001      45.0       1.53  ...  317.080258  148.144044     800
3      1998      47.0       1.45  ...  317.080258  148.144044     900
4      1992      45.0       1.45  ...  241.000000   49.000000    1200

[5 rows x 14 columns]
     BC    Hb  MCHC    Sexe
0  4.36  84.6  34.1    Male
1  6.78  37.4  34.6  Female
2  14.8    37  34.8    Male
3  12.8  32.8  33.2    Male
4   6.6   9.6  31.5  Female


In [154]:
df_fin = pd.get_dummies(df_num, columns=["Sexe"])
print(df_fin.head())
print(df_fin.dtypes)
print(df_fin[df_fin.isnull().any(axis=1)])
#df_fin.dropna()

   năm sinh  Cân nặng  Chiều cao  ...  Weight  Sexe_Female  Sexe_Male
0      1982      63.0       1.50  ...    3600            0          1
1      1999      46.0       1.59  ...     350            1          0
2      2001      45.0       1.53  ...     800            0          1
3      1998      47.0       1.45  ...     900            0          1
4      1992      45.0       1.45  ...    1200            1          0

[5 rows x 16 columns]
năm sinh         int64
Cân nặng       float64
Chiều cao      float64
BMI            float64
Tuần thai      float64
CHBC           float64
Hct            float64
MCV            float64
MCH            float64
Mao mạch       float64
BPD            float64
AC             float64
FL             float64
Weight           int64
Sexe_Female      uint8
Sexe_Male        uint8
dtype: object
Empty DataFrame
Columns: [năm sinh, Cân nặng, Chiều cao, BMI, Tuần thai, CHBC, Hct, MCV, MCH, Mao mạch, BPD, AC, FL, Weight, Sexe_Female, Sexe_Male]
Index: []


In [155]:
print(df_fin.dtypes)
df_fin.head()

năm sinh         int64
Cân nặng       float64
Chiều cao      float64
BMI            float64
Tuần thai      float64
CHBC           float64
Hct            float64
MCV            float64
MCH            float64
Mao mạch       float64
BPD            float64
AC             float64
FL             float64
Weight           int64
Sexe_Female      uint8
Sexe_Male        uint8
dtype: object


,năm sinh,Cân nặng,Chiều cao,BMI,Tuần thai,CHBC,Hct,MCV,MCH,Mao mạch,BPD,AC,FL,Weight,Sexe_Female,Sexe_Male
0,1982,63.0,1.50,28.00,11.4,12.60,36.9,28.8,34.1,100.0,92.000000,350.000000,74.000000,3600,0,1
1,1999,46.0,1.59,28.90,24.0,4.61,12.9,81.2,28.1,75.0,106.981032,317.080258,148.144044,350,1,0
2,2001,45.0,1.53,19.00,25.0,4.45,12.9,83.1,28.9,71.0,106.981032,317.080258,148.144044,800,0,1
3,1998,47.0,1.45,22.35,26.0,3.62,10.9,90.4,30.0,85.0,106.981032,317.080258,148.144044,900,0,1
4,1992,45.0,1.45,21.40,26.0,4.17,30.3,72.8,23.0,86.0,70.000000,241.000000,49.000000,1200,1,0


In [ ]:
# Scale all numeric column 
import numpy as np
from sklearn.preprocessing import MinMaxScaler

num_cols = df_fin.columns[df_fin.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
scaler = MinMaxScaler()
df_fin_scaled = df_fin
scaler.fit_transform(df_fin_scaled[num_cols])
col_one_list = df_fin['năm sinh'].tolist()
col_one_list

Move target column to the end of data frame

In [134]:
x = df_fin_scaled.pop('Weight') # remove column b and store it in df1
df_fin_scaled['Weight']=x
print( df_fin_scaled.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   năm sinh     2517 non-null   float64
 1   Cân nặng     2517 non-null   float64
 2   Chiều cao    2517 non-null   float64
 3   BMI          2517 non-null   float64
 4   Tuần thai    2517 non-null   float64
 5   BC           2517 non-null   float64
 6   CHBC         2517 non-null   float64
 7   Hct          2517 non-null   float64
 8   Hb           2517 non-null   float64
 9   MCV          2517 non-null   float64
 10  MCH          2517 non-null   float64
 11  MCHC         2517 non-null   float64
 12  Mao mạch     2517 non-null   float64
 13  BPD          2517 non-null   float64
 14  AC           2517 non-null   float64
 15  FL           2517 non-null   float64
 16  Sexe_Female  2517 non-null   float64
 17  Sexe_Male    2517 non-null   float64
 18  Weight       2517 non-null   float64
dtypes: flo

Split test and train dataset

In [135]:
array = df_fin_scaled.values
X = array[:,0:17]
Y = array[:,18]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

Build models

In [121]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


num_folds = 10
kfold = KFold(n_splits=10, shuffle=True, random_state=7)

models = []
models.append(('Ridge regression', Ridge()))
models.append(('Lasso regression', Lasso()))
models.append(('ElasticNet', ElasticNet()))
models.append(('DecisionTreeRegressor', DecisionTreeRegressor()))
models.append(('SVR', SVR()))

scoring = 'neg_mean_squared_error'
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

Ridge regression: -0.005311 (0.001154)
Lasso regression: -0.009163 (0.001697)
ElasticNet: -0.009163 (0.001697)
DecisionTreeRegressor: -0.010302 (0.001738)
SVR: -0.005168 (0.001083)


Support vector machine regression is the best model on Traing set.

In [182]:
model = SVR()
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

scaler = MinMaxScaler()
x = df_fin["Weight"].values
scaler.fit_transform(x.reshape(-1, 1))
Y_validation_inv = scaler.inverse_transform(Y_validation.reshape(1, -1))[0,:].tolist()
Y_predictions_inv = scaler.inverse_transform(predictions.reshape(1, -1))[0,:].tolist()
res = pd.DataFrame(list(zip(Y_validation_inv, Y_predictions_inv)), columns =['Y_validation', 'predictions']) 

res


,Y_validation,predictions
0,3200.0,3319.440453
1,2600.0,2306.730148
2,2900.0,2920.718586
3,2900.0,3183.866354
4,3700.0,3265.377829
...,...,...
499,3500.0,3291.938447
500,2100.0,2743.498773
501,2900.0,2813.069680
502,3500.0,3226.847267


In [188]:
model = SVR()
model.fit(X_train, Y_train)
predictions = model.predict(X_train)

scaler = MinMaxScaler()
x = df_fin["Weight"].values
scaler.fit_transform(x.reshape(-1, 1))
Y_validation_inv = scaler.inverse_transform(Y_train.reshape(1, -1))[0,:].tolist()
Y_predictions_inv = scaler.inverse_transform(predictions.reshape(1, -1))[0,:].tolist()
res = pd.DataFrame(list(zip(Y_validation_inv, Y_predictions_inv)), columns =['Y_validation', 'predictions']) 

res

,Y_validation,predictions
0,3700.0,3533.748487
1,2600.0,2773.481207
2,3600.0,3465.284793
3,3000.0,2640.810633
4,2200.0,3201.594924
...,...,...
2008,3000.0,3159.678176
2009,3700.0,3348.246824
2010,3300.0,3283.585549
2011,3700.0,3090.233104


In [189]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(Y_validation_inv,Y_predictions_inv) )


320.5817305783175